In [30]:
from typing import Union, Final, Tuple


### 전역 상수 선언
MAX_LEVERAGE = 125
MIN_LEVERAGE = 2
BUY_TYPE:Tuple[int, str] = (1, 'BUY')
SELL_TYPE:Tuple[int, str] = (2, 'SELL')

def __validate_params_position(position:Union[int, str]) -> Union[int, str]:
    # position타입이 문자형인 경우
    if isinstance(position, str):
        # 대문자로 변환
        position = position.upper()
        # 각 타입의 index 1을 set으로 구성 및 포함여부 확인
        if position not in {BUY_TYPE[1], SELL_TYPE[1]}:
            raise ValueError(f"position 입력 오류: {position}")

    # position을 정수형으로 변환 (숫자이거나 문자열에서 변환된 경우)
    elif isinstance(position, int):
        # 각 타입의 index 0을 set으로 구성 및 포함여부 확인
        if position not in {BUY_TYPE[0], SELL_TYPE[0]}:
            raise ValueError(f"position 입력 오류: {position}")
    
    # params position 입력타입 오류시
    else:
        raise ValueError(f"position은 int 또는 str만 입력 가능: {type(position)}")
    return position

def __validate_params_leveragse(leverage:int) -> int:
    if not isinstance(leverage, int):
        raise ValueError(f'leverage 타입 입력 오류: {type(leverage)}')
    if MAX_LEVERAGE < leverage:
        raise ValueError(f'leverage는 {MAX_LEVERAGE}를 초과할 수 없음: {leverage}')
    if MIN_LEVERAGE > leverage:
        raise ValueError(f'leverage는 최소 {MIN_LEVERAGE} 이상이어야 함: {leverage}')
    return leverage
        
    
    

def calculate_imr(leverage:int) -> float:
    """
    IMR(Initial Margin Ratio) 초기 증거금 비율을 계산한다.

    Args:
        leverage (int): 적용할 레버리지값

    Raises:
        ValueError: leverage가 int가 아닐경우
        ValueError: leverage가 MAX_LEVERAGE(125배)를 초과할 경우

    Returns:
        float: IMR값
        
    Example:
        leverage = 5
        imr = calculate_imr(leverage)
        print(imr)
    """
    leverage = __validate_params_leveragse(leverage)
    return 1/leverage

def calculate_initial_margin(quantity:Union[float, int], entry_price:Union[float, int], imr:float) -> float:
    """
    Binance Futures 거래시 초기 마진값을 계산한다. Spot 거래에서는 사용하지 않는다.
    
    Args:
        quantity (Union[float, int]): 진입 수량
        entry_price (Union[float, int]): 진입 가격
        imr (float): 초기 마진 비율
            >> IMR(Initial Margin Ratio) 초기 증거금 비율
            >> calculate_imr 함수로 계산가능.

    Return:
        float: 초기 마진값 (USDT 기준)
        
    Notes:
        fee는 반영이 안되어 있음.
    
    Example:
        leverage = 5
        quantity = 10
        entry_price = 4.5
        imr = calculate_imr(leverage)
        
        initial_margin = calculate_initial_margin(quantity, entry_price, imr)
    """
    return quantity * entry_price * imr

def calculate_pnl(
    position: Union[int, str], 
    entry_price: Union[float, int], 
    exit_price: Union[float, int], 
    quantity: Union[float, int]
) -> float:
    """
    포지션(롱/숏)에 따른 손익(PnL)을 계산하는 함수.

    Args:
        position (Union[int, str]): 포지션 (1 또는 'BUY' / 2 또는 'SELL')
        entry_price (Union[float, int]): 진입 가격
        exit_price (Union[float, int]): 종료 가격
        quantity (Union[float, int]): 계약 수량

    Returns:
        float: 손익 (PnL)

    Raises:
        ValueError: position이 올바르지 않은 값일 경우 예외 발생
        
    Example:
        position = 1
        entry_price = 1.2
        exit_price = 1.3
        quantity = 10
        
        pnl = calculate_pnl(position, entry_price, exit_price, quantity)
    """
    validated_position = __validate_params_position(position=position)
    # PnL 계산 (롱 포지션인지, 숏 포지션인지 판별)
    if validated_position in {BUY_TYPE[0], BUY_TYPE[1]}:  # 롱(매수) 포지션
        return (exit_price - entry_price) * quantity
    else:  # 숏(매도) 포지션
        return (entry_price - exit_price) * quantity

def calculate_roi_1(initial_margin:float, pnl:float) -> float:
    """
    투자 수익률(ROI, Return on Inverstment) 계산 (공식 1)
    
    Args:
        initial_margin (float): 초기증거금 비율
            >> calculate_imr(leverage)를 이용하여 계산
        pnl (float): 손익금액
            >> calculate_pnl(position, entry_price, exit_price, quantity)를 이용하여 산출
    
    Resturn:
        float: 투자 수익률
    
    Notes:
        ROI 계산 공식중 하나를 적용
    
    Example:
        entry_price = 1.2
        quantity = 10
        initial_margin = calculate_pnl(quantity, entry_price)
        
        position = 1
        exit_price = 1.3
        pnl = calculate_pnl(position, entry_price, exit_price, quantity)
    
        roi = calculate_roi_1(initial_margin, pnl)
    """
    return pnl / initial_margin

def calculate_roi_2(entry_price:float, exit_price:float, position:Union[int, str], imr:float) -> float:
    """
    투자 수익률(ROI, Return on Inverstment) 계산 (공식 2)

    Args:
        entry_price (float): 진입 가격
        exit_price (float): 종료 가격
        position (Union[int, str]): 포지션
        imr (float): 초기 마진값

    Returns:
        float: 투자 수익률
        
    Notes:
        ROI 계산 공식중 하나를 적용
        
    Example:
        entry_price = 1.2
        exit_price = 1.3
        position = 1
        leverage = 5
        
        imr = calculate_imr(leverage)
        
        roi = calculate_roi_2(entry_price, exit_price, position, imr)
        
    """
    validated_position = __validate_params_position(position=position)
    side = 1 if validated_position in SELL_TYPE else -1
    return side * ((1-(exit_price/entry_price))/imr)

def calculate_target_price(entry_price:float, target_roi:float, leverage:int, position:Union[float, str]) -> float:
    """
    수익률을 반영하여 목표 단가를 산출한다.

    Args:
        entry_price (float): 진입가
        target_roi (float): 목표 수익률
        leverage (int): 레버리지
        position (Union[float, str]): 포지션

    Returns:
        float: 목표 단가
    
    Notes:
        목표 수익률에 다른 포지션 종료 단가를 계산한다.
    
    Example:
        entry_price = 1.2
        target_roi = 0.5  (50%)
        leverage = 5
        position = 1

        target_price = calculate_target_price(entry_price, target_roi, leverage, position)
    """
    
    validated_leverage = __validate_params_leveragse(leverage)
    validated_position = __validate_params_position(position)
    if validated_position in BUY_TYPE:
        return entry_price * ((target_roi/validated_leverage)+1)
    else:
        return entry_price * (1-(target_roi/leverage))
    

In [34]:
entry_price = 1.2
exit_price = 1.3
quantity = 10
leverage = 125
position = 1

In [35]:
imr = calculate_imr(leverage)
margin = calculate_initial_margin(quantity, entry_price, imr)
pnl = calculate_pnl(position, entry_price, exit_price, quantity)
roi_1 = calculate_roi_1(margin, pnl)
roi_2 = calculate_roi_2(entry_price, exit_price, position, imr)
target_price = calculate_target_price(entry_price, 2, leverage, position)

print(f'IMR: {imr}')
print(f'initial_margin: {margin}')
print(f'pnl: {pnl}')
print(f'roi_1: {roi_1}')
print(f'roi_2: {roi_2}')
print(f'target_price: {target_price}')

IMR: 0.008
initial_margin: 0.096
pnl: 1.0000000000000009
roi_1: 10.416666666666675
roi_2: 10.416666666666686
target_price: 1.2192


In [29]:
entry_price * ((0.1/leverage)+1)

1.224